In [211]:
from abc import ABCMeta, abstractmethod
import pandas_datareader as pdr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from yahoo_fin import stock_info as si
from scipy import optimize
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go

## Abstract Classes

In [212]:
class DataHandler(metaclass=ABCMeta):
    
    @abstractmethod
    def get_data(self):
        raise NotImplementedError("Should implement get_data()!")
    
    @abstractmethod
    def get_symbol(self):
        raise NotImplementedError("Should implement get_symbol()!")

class Strategy(metaclass=ABCMeta):

    @abstractmethod
    def generate_signals(self):
        raise NotImplementedError("Should implement generate_signals()!")

class Portfolio(metaclass=ABCMeta):
    @abstractmethod
    def generate_positions(self):
        raise NotImplementedError("Should implement generate_positions()!")

    @abstractmethod
    def backtest_portfolio(self):
        raise NotImplementedError("Should implement backtest_portfolio()!")

class graphics(metaclass=ABCMeta):
    @abstractmethod
    def plot_graph(self):
        raise NotImplementedError("Should implement plot_graph()!")

## DataHandler

In [213]:
class My_data(DataHandler):    
    def __init__(self):
        pass
    
    def screening(self):
        
        # screen for volume
        volume_data = pd.read_csv(r'D:\Directory\Current Projects\Option Trading Strategy\volume_data.csv')
        volume_data = volume_data.sort_values(by=['Avg. Volume'], ascending=False)
        vol_symbol = np.array([x for x in volume_data.head(30)['index']])
        
        # get the data for the 30 most liquid stocks
        start = datetime.datetime(2020, 1, 1)
        end = datetime.datetime(2022, 2, 1)
        data = pdr.get_data_yahoo(vol_symbol, start=start, end=end)['Adj Close']
        
        # get the 5 stocks with the highest sharpe ratio
        returns = data.pct_change()
        mean_returns = returns.mean()
        stds = returns.std()
        
        sharpe = pd.DataFrame((mean_returns - 0.02)/stds, index=data.columns, columns=['sharpe'])
        sharpe = sharpe.sort_values(by=['sharpe'], ascending=False)
        symbol = np.array([x for x in sharpe.head(5).index])
        
        return symbol, data[symbol]
        
    def get_data(self):
        return self.screening()[1]
    
    def get_symbol(self):
        return self.screening()[0]

## Starategy

In [214]:
class My_startegy(Strategy):    
    def __init__(self, symbol, data, short_window=8, long_window=24):
        self.symbol = symbol
        self.data = data
        self.short_window = short_window
        self.long_window = long_window
    
    def Exponential_Moving_Averages(self, short_window=8, long_window=24):
        
        short_exp_mavg = self.data.rolling(self.short_window).mean()
        long_exp_mavg = self.data.rolling(self.long_window).mean()
        
        return short_exp_mavg, long_exp_mavg
    
    def generate_signals(self):
        signals = pd.DataFrame(index=self.data.index, columns=self.data.columns).fillna(0)
        
        # Moving average
        short_exp_mavg, long_exp_mavg = self.Exponential_Moving_Averages()
        
        signals[self.short_window:] = np.where(short_exp_mavg[self.short_window:] > long_exp_mavg[self.short_window:], 1.0, 0.0)
        
        return signals

## Portfolio

In [215]:
class My_Portfolio(Portfolio):
    
    def __init__(self, symbol, data, signals, initial_capital=1000000):
        self.symbol = symbol        
        self.data = data
        self.signals = signals
        self.initial_capital = float(initial_capital)
    
    def generate_positions(self):
        positions = pd.DataFrame(index=signals.index).fillna(0.0)
        positions[self.symbol] = 100*signals.diff()
        return positions
    
    def backtest_portfolio(self):
        portfolio = self.generate_positions()*self.data
        
        portfolio['holdings'] = (self.generate_positions()*self.data).sum(axis=1)
        portfolio['cash'] = self.initial_capital - (self.generate_positions()*self.data).sum(axis=1).cumsum()
        portfolio['total'] = portfolio['cash'] + portfolio['holdings']
        portfolio['returns'] = portfolio['total'].pct_change().fillna(0)
        return portfolio


In [217]:
class My_graphic(graphics):
    def __init__(self, symbol, data, signals, short_exp_mavg, long_exp_mavg, holdings):
        self.symbol = symbol        
        self.data = data
        self.signals = signals
        self.short_exp_mavg = short_exp_mavg
        self.long_exp_mavg = long_exp_mavg
        self.holdings = holdings
    
    def plot_graph(self):
        
        app = JupyterDash(__name__)
        colors = {"background": "#191724", "text": "#bcc3cd"}
        
        graph_type = ['Chart', 'Equity Curve']

        app.layout = html.Div([
                        html.Div([
                            html.Div([dcc.Dropdown(id="graph_type", options=graph_type, multi=False, value='Chart', style={'width': '48%'})], style={'width': '48%', 'display': 'inline-block'}),
                            html.Div([dcc.Dropdown(id="select_stock", options=self.symbol, multi=False, value='TSLA', style={'width': '48%'})], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
                        ]),
                        html.Div([dcc.Graph(id="my_graph", figure={})])    
                    ])

        @app.callback(Output(component_id="my_graph", component_property="figure"),
                      [Input("graph_type", "value"), Input("select_stock", "value")])
        

        def update_output(argument2, argument1):
            
            if (argument2 == 'Chart'):

                fig = go.Figure()

                # stock price
                fig.add_trace(go.Scatter(x=self.data.index, y=self.data[argument1].values, mode='lines', name=argument1, line_color='#abc1c4', line_width=1.5, fillcolor='rgba(23,48,102,0.5)', fill='tonexty'))


                # Exponential_Moving_Averages
                fig.add_trace(go.Scatter(x=self.data.index, y=short_exp_mavg[argument1].values, mode='lines', name='8 days EMA', line_color='#32CD32', line_width=1))
                fig.add_trace(go.Scatter(x=self.data.index, y=long_exp_mavg[argument1].values, mode='lines', name='24 days EMA', line_color='#DC143C', line_width=1))

                # Signals
                fig.add_trace(go.Scatter(name='Buy', x=signals[argument1].diff()[signals[argument1].diff() == 1].index, 
                                         y=self.data[argument1][signals[argument1].diff() == 1], mode="markers",
                                         marker_symbol="triangle-up", marker_line_color="#32CD32", 
                                         marker_color="#32CD32", marker_line_width=2, marker_size=10, showlegend=False))
                fig.add_trace(go.Scatter(name='Sell', x=signals[argument1].diff()[signals[argument1].diff() == -1].index, 
                                         y=self.data[argument1][signals[argument1].diff() == -1], mode="markers",
                                         marker_symbol="triangle-down", marker_line_color="#B22222", 
                                         marker_color="#B22222", marker_line_width=2, marker_size=10, showlegend=False))

                fig.update_xaxes(showline=True, linewidth=2, linecolor='#2D334A', gridcolor='#55535e')
                fig.update_yaxes(showline=True, linewidth=2, linecolor='#2D334A', gridcolor='#55535e')
                fig.update_layout(
                    plot_bgcolor=colors["background"],
                    paper_bgcolor=colors["background"],
                    font_color=colors["text"],
                    height = 550,
                    margin=dict(l=20, r=20, t=20, b=20) #2b477d
                )
            
                return fig
        
            else:
                
                fig = go.Figure()

                # stock price
                fig.add_trace(go.Scatter(x=self.data.index, y=self.holdings.values, mode='lines', line_color='#abc1c4', line_width=1.5, fillcolor='rgba(23,48,102,0.5)', fill='tonexty'))
                
                fig.update_xaxes(showline=True, linewidth=2, linecolor='#2D334A', gridcolor='#55535e')
                fig.update_yaxes(showline=True, linewidth=2, linecolor='#2D334A', gridcolor='#55535e')
                fig.update_layout(
                    plot_bgcolor=colors["background"],
                    paper_bgcolor=colors["background"],
                    font_color=colors["text"],
                    height = 550,
                    margin=dict(l=20, r=20, t=20, b=20) #2b477d
                )
            
                return fig
        

        # Run app and display result inline in the notebook
        app.run_server(mode='inline', port=8059)

In [219]:
if __name__ == "__main__":
    
    dataHandler = My_data()
    data = dataHandler.get_data()
    
    rfs = My_startegy(dataHandler.get_symbol(), data, short_window=8, long_window=24)
    short_exp_mavg, long_exp_mavg = rfs.Exponential_Moving_Averages()
    signals = rfs.generate_signals()
    
    
    portfolio = My_Portfolio(dataHandler.get_symbol(), data, signals, initial_capital=500000.0)
    returns = portfolio.backtest_portfolio()
    
    
    graphics = My_graphic(dataHandler.get_symbol(), data, signals, short_exp_mavg, long_exp_mavg, returns['total'])
    fig = graphics.plot_graph()
    
    display(returns)

C:\Users\mhaim\anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



,TSLA,NCLH,OXY,CCL,MRO,holdings,cash,total,returns
Date,,,,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,0.000000,500000.000000,500000.000000,0.000000
2020-01-03,0.0,0.0,0.0,0.000000,0.0,0.000000,500000.000000,500000.000000,0.000000
2020-01-06,0.0,0.0,0.0,0.000000,0.0,0.000000,500000.000000,500000.000000,0.000000
2020-01-07,0.0,0.0,0.0,0.000000,0.0,0.000000,500000.000000,500000.000000,0.000000
2020-01-08,0.0,0.0,0.0,0.000000,0.0,0.000000,500000.000000,500000.000000,0.000000
...,...,...,...,...,...,...,...,...,...
2022-01-26,0.0,0.0,0.0,-2029.000092,0.0,-2029.000092,572884.174013,570855.173922,0.000000
2022-01-27,0.0,0.0,0.0,0.000000,0.0,0.000000,572884.174013,572884.174013,0.003554
2022-01-28,0.0,0.0,0.0,0.000000,0.0,0.000000,572884.174013,572884.174013,0.000000
